In [ ]:
### TODOs ###

# TODOs: Commonsense Query and Concept Expansion: Topics, Concepts, Synonyms
# TODOs: Domain Restrict. Polarising social and political debate (Class labelling) only for higher-quality
# TODO: Enhance Stance Module; Determine stance over entire argument. Only implicate stance for Noun
# TODOs: Targeted Retrieval with Semantic Graphs
# TODOs: Multi-Field Search

# DONE: Mine Args
# DONE: Mine Counters
# DONE: Add Concepts
# DONE: Prior tokenization and sentence segmentation to speed processing
# DONE: Adu, Counter + KP Extraction as 'Argument Mining' preprocessing module
# DONE: Manage Duplicate Keywords
# DONE: Sentential Ranking
# DONE: Include Topic Label
# DONE: Include Concept Label
# DONE: Add News

# TODOs: (1) coverage of topic signature words in the input statement; (2) a weighted summation of the coverage of n-grams in the argu- ment4; (3) the magnitude of stance score, where we keep the passages of the same polarity as the argument; (4) content word overlap with the argument; and (5) coverage of topic signature words in the argument.

In [18]:
import os
root = "/Users/joshua.sheppard/PycharmProjects/countaBot"
os.chdir(root)
print(os.getcwd())

### INIT LOGGING ###
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ARGUMENT-EXTRACTOR")

### NLP FUNCTIONS ###
from src.utils_.utils import tokeniser, sentences_segment

/Users/joshua.sheppard/PycharmProjects/countaBot


In [20]:
### LOAD DATASETS ###
import json
import os
import random

print(os.getcwd())
args = [json.loads(ln) for ln in open("./src/data/cmv_processed.jsonl")]
topics = [json.loads(ln) for ln in open("./src/data/argument_topic_concept.jsonl")]
concepts = [json.loads(ln) for ln in open("./src/data/argument_concept.jsonl")]

/Users/joshua.sheppard/PycharmProjects/countaBot


In [21]:
len(topics), len(concepts), len(args)

(5990, 5990, 10303)

In [23]:
### ASSERT BLANKS ###
args_ = [json.loads(ln)["argument"]["argument"] for ln in open("./src/data/cmv_processed.jsonl")]
ids = [json.loads(ln)["id"] for ln in open("./src/data/cmv_processed.jsonl")]

for j, k in zip(args_, ids):
    if j == "":
        print("blanks", j, k)

blanks  t3_3cm6jy
blanks  t3_1egv4k
blanks  t3_1egv4k
blanks  t3_5wjdve


In [24]:
### INSPECT ARG ###
import random
sample = random.randint(0, 99)

arg = args[sample]["argument"]["argument"]
claim = args[sample]["claim"]

print(sample, "\n")
print(claim, "\n")
print(arg, "\n")

99 

rewinding time is the best superpower for daily life 

a topic that ive had fun discussing is to imagine what could be done with a superpower. with so many choices and so many implications for each power this little game can spark long conversations on how each power would affect our daily lives. from all of these discussions ive come away with the view that one power is better than all others by the most metrics a power that i like to call rewind.disclaimer if youve never found yourself wondering what your life could be like with superpowers and have no interest in starting then this topic is definitely not for you. the topic is one massive hypothetical so that better be your thing ptldr because damn! i wrote way too much to ask you to read it all 



In [26]:
### EXTRACTORS ###
from src.utils_.keyphrase_extraction import yake_extract_keyphrase, summa_extract_keyphrase

test = "Brazil's minimum income has increasingly been accepted."
ev_kp = yake_extract_keyphrase(test)
ev_kp_ = summa_extract_keyphrase(test)

test_2 = " "
ev_kp_2 = yake_extract_keyphrase(test_2)
ev_kp_2_ = summa_extract_keyphrase(test_2)

print(ev_kp)
print(ev_kp_)

# Can Handel Blanks
print(ev_kp_2)
print(ev_kp_2_)

### KEYBERT ###
from keybert import KeyBERT

kb = KeyBERT()
def extract_keyphrase(doc, n_gram=3, n_kp=3, use_mmr="False", use_maxsum="False"):
    kp = kb.extract_keywords(doc, keyphrase_ngram_range=(1, 4), stop_words='english', use_mmr=True, diversity=0.5)

    return [i[0] for i in kp]


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


['Brazil minimum income', 'minimum income has increasingly', 'Brazil minimum', 'increasingly been accepted', 'minimum income']
['minimum']
[]
[]


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [32]:
from tqdm.notebook import tqdm
from src.utils_.word_net_expansion import expand_query
import multiprocessing
import json
import time

# Disable Huggingface Logging
import os
os.chdir(home)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

topic_ids = [json.loads(ln)["id"] for ln in open("./src/data/argument_topic_concept.jsonl")]
concept_ids = [json.loads(ln)["id"] for ln in open("./src/data/argument_concept.jsonl")]

# path = "/Users/joshua.sheppard/PycharmProjects/countaBot/src/retriever/"
# os.chdir(path)
from src.detection.stance_classifier import sentence_stance
from src.detection.stance_classifier import sentence_stance, compare_stance

# Where notion == topic or concept
def get_notion(notions_ids, notions_lst, arg_id, label):
    notion_id = notions_ids.index(arg_id)
    notion = notions_lst[notion_id][label]
    return str(notion) if notion else None

# Extract Argument Discourse as Sentences, Keyphrases, Topics and Concepts
def extract_adus(arg_):

    id_ = arg_["id"]
    arg = arg_["argument"]["argument"]

    topic = get_notion(topic_ids, topics, id_, "topic_label")
    concept = get_notion(concept_ids, concepts, id_, "concept_label")

    adu_sents = sentences_segment(arg)

    extract_adus = []
    for _ in adu_sents:

        # Limit irrelevant sentences
        if len(tokeniser(_)) <= 5:
            continue

        kp = extract_keyphrase(_)
        #kp = yake_extract_keyphrase(_)

        aspect = " " if kp == [] else kp[0]

        try:
            stance = sentence_stance(_, aspect)
        except:
            stance = " "

        adu = {"sentence": _, "kp": [i.lower() for i in kp], "stance": stance, "aspect": aspect, "topic": topic, "concept": concept}

        extract_adus.append(adu)

    claim = arg_["claim"]
    claim_kp = yake_extract_keyphrase(claim)
    claim_adu = {"sentence": claim, "kp": [i for i in claim_kp]}

    return ({
        "id": id_,
        "claim": claim_adu,
        "argument": [i for i in extract_adus]
    })

mined_args = []

with multiprocessing.Pool(8) as pool:
    with tqdm(total=(len(args)), position=0, leave=True) as pbar:
        for arg in args:
            mined_args.append(extract_adus(arg))
            pbar.update()

  0%|          | 0/10303 [00:00<?, ?it/s]

In [33]:
# STORE DEEP-COPY
import copy
mined_args_ = copy.deepcopy(mined_args)

In [35]:
# mined_args
print(len(mined_args))

import random
#_ = random.randint(0, len(sample))
example = mined_args[1]
example

10303


{'id': 't3_30oi71',
 'claim': {'sentence': 'we should strengthen the traditional safety net rather than replace it with basic income',
  'kp': ['strengthen the traditional safety',
   'traditional safety net',
   'basic income',
   'strengthen the traditional',
   'traditional safety']},
 'argument': [{'sentence': 'section i why is basic income increasingly popular?',
   'kp': ['basic income increasingly popular',
    'section basic income',
    'basic',
    'increasingly',
    'section'],
   'stance': 'PRO',
   'aspect': 'basic income increasingly popular',
   'topic': 'incomes policy',
   'concept': 'increase in accumulated other comprehensive income'},
  {'sentence': 'basic income is a policy that has broad support from both the progressive left and libertarian right.',
   'kp': ['basic income',
    'basic income policy',
    'basic income policy broad',
    'income policy broad support',
    'support progressive left libertarian'],
   'stance': 'PRO',
   'aspect': 'basic income',
 

In [37]:
len(mined_args_)

10303

In [ ]:
### COUNTER-ARGS ###
def extract_counters(arg_):
    id_ = arg_["id"]
    counter = arg_["tgt_counter"]["tgt_counter"]

    counter_sents = sentences_segment(counter)

    extract_counters = []
    for _ in counter_sents:
        if len(tokeniser(_)) <= 5:
            continue

        kp = extract_keyphrase(_)
        #kp = yake_extract_keyphrase(_)

        aspect = " " if kp == [] else kp[0]

        try:
            stance = sentence_stance(_, aspect)
        except:
            stance = " "

        counter_unit = {"sentence": _, "kp": [i for i in kp], "stance": stance, "aspect": aspect}

        extract_counters.append(counter_unit)

    return ({
        "id": id_,
        "tgt_counter": [i for i in extract_counters]
    })

mined_counters = []

sample = args
with multiprocessing.Pool(8) as pool:
    with tqdm(total=(len(args)), position=0, leave=True) as pbar:
        for arg in args:
            mined_counters.append(extract_counters(arg))
            pbar.update()

  0%|          | 0/10303 [00:00<?, ?it/s]

In [15]:
import copy
mined_counters_ = copy.deepcopy(mined_counters)

In [13]:
# mined_args
print(len(mined_counters))

import random
_ = random.randint(0, len(sample))
example = mined_counters[_]
example

10303


{'id': 't3_2htk52',
 'tgt_counter': [{'sentence': 'commercial dry dog food is full of additives preservatives rendered animal products meat not inspected as safe for humans etc.',
   'kp': ['commercial dry dog food',
    'additives preservatives rendered animal',
    'preservatives rendered animal products',
    'rendered animal products meat',
    'commercial dry dog'],
   'stance': 'PRO',
   'aspect': 'commercial dry dog food'},
  {'sentence': 'some people think dog kibble isnt even healthy for the dog some of the chemicals theyre worried about include bht and ethoxyquin.',
   'kp': ['include bht and ethoxyquin',
    'kibble isnt even healthy',
    'chemicals theyre worried',
    'theyre worried about include',
    'worried about include bht'],
   'stance': 'PRO',
   'aspect': 'include bht and ethoxyquin'},
  {'sentence': 'if the child ate one or two pieces once thats not the end of the world.',
   'kp': ['child ate', 'end of the world', 'world', 'child', 'ate'],
   'stance': 'NEUTRA

In [17]:
len(mined_args), len(mined_counters)

(10303, 10303)

In [16]:
len(mined_args_), len(mined_counters_)

(10303, 10303)

In [19]:
import os
print(os.getcwd())

/Users/joshua.sheppard/PycharmProjects/countaBot


In [20]:
file_name = "cmv_argument_extraction"
fout = open(f"./src/data/{file_name}.jsonl", "w")

# Deep_copies
mined_args_ = copy.deepcopy(mined_args)
mined_counters_ = copy.deepcopy(mined_counters)

with tqdm(total=(len(mined_args_))) as pbar:
    with fout:
        for mined_arg, mined_counter in zip(mined_args_, mined_counters_):
            # Extended pre-formatted mined object
            # mined_arg["original_post"] = original_post["argument"]
            mined_arg["tgt_counter"] = [_ for _ in mined_counter["tgt_counter"]]

            fout.write(json.dumps(mined_arg))

            fout.write("\n")
            pbar.update()

logger.info(f"[{len(mined_args_)} Data Stored as {file_name}.jsonl]")

  0%|          | 0/10303 [00:00<?, ?it/s]

INFO:ARGUMENT-EXTRACTOR:[10303 Data Stored as cmv_argument_extraction.jsonl]


In [21]:
### EVALUATE OUTPUT ###
train = [json.loads(ln) for ln in open(f"../data/{file_name}.jsonl", "r")]

In [22]:
len(train)

10303

In [28]:
_ = random.randint(0, len(train))
print(train[_]["id"], "\n")
print(train[_]["argument"], "\n")
print(train[_]["tgt_counter"], "\n")

t3_2dqwmd 

[{'sentence': 'Let me start off by saying that Im a gay guy.', 'kp': ['gay guy', 'guy', 'start', 'gay'], 'stance': 'NEUTRAL', 'aspect': 'gay guy', 'topic': None, 'concept': 'gay guy'}, {'sentence': 'Im not really viewing this from the outside in though perhaps that perspective is what I need.Im nineteen years old now coming to realization with my sexuality when I had just turned thirteen years old.', 'kp': ['need.Im nineteen years', 'turned thirteen years', 'need.Im nineteen', 'coming to realization', 'turned thirteen'], 'stance': 'NEUTRAL', 'aspect': 'need.Im nineteen years', 'topic': None, 'concept': 'need.Im nineteen years'}, {'sentence': 'I was a loner who read wrote and gamed in my spare time rather than hang out with friends.', 'kp': ['loner who read', 'read wrote', 'wrote and gamed', 'spare time', 'friends'], 'stance': 'CON', 'aspect': 'loner who read', 'topic': None, 'concept': 'loner who read'}, {'sentence': 'People instantly expected me to talk with a lisp walk wi

In [ ]:
# for i, j in zip(retrieved_ranked, sample):
#     # Add counter to the dictionary (implicitly, i)
#     i["counter"] = j["counter"]
#     fout.write(json.dumps(i))
#     fout.write("\n")

In [ ]:
# Working Loop
# for i in mined_counters_:
#     test = {
#         "test": [j for j in i["counter"]]
#     }

In [ ]:
# BATCH LOADING

# STEPS = 10
# STEP = max(int(len(SAMPLE) / STEPS), 1)
# BATCHES = [sample[i:i + STEP] for i in range(0, len(SAMPLE), STEP)]
#
# mined_counters = []
# for idx, batch in enumerate(BATCHES):
#     print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(BATCHES)) + '-' * 25)
#
#     with multiprocessing.Pool(8) as pool:
#         with tqdm(total=(len(batch))) as pbar:
#             for counter in batch:
#                 mined_counters.append(extract_counters(counter))
#                 pbar.update()